# The Introduction

[Bitcoin](https://bitcoin.org/), [ethereum](https://ethereum.org/) and other blockchains are developed so quickly during 2010s. Blockchains provide a decentrilized and auditable infrastructure to the users, allowing all members to maintain their own copies of the same ledger and ensure that the copies are all synchronized. Blockchains can be used as a ledger, a global computer or other state machines and it drives many on-chain applications.

One of the key challenges of this decentralized design is incentives. The incentive model of a blockchain is to calculating the system income (including inflation and transaction fee) and design the redistribution of the income. Another article of mine can be read (TODO: add link here) if you want to know more about the whole picture of fundamental mechanism of blockchains. Simply speaking, ignoring the difference among various consensus algrithms, the responsible participants of on-chain governance should be motivated and [bitcoin](https://bitcoin.org/), [ethereum](https://ethereum.org/), [eos](https://eos.io/) and other blockchains proposed various good solutions.

Actually on-chain behavior should be rewarded by on-chain tokens, which means, the incentive model is based on a basic token model of a blockchain. Nowadays, most of blockchains use single token model. The only token in the single token model may be used as a governance token as well as a utility token and more or less, it worked pretty well.

Blockchains like [neo](https://neo.org/) and [ontology](https://ont.io/) use a dual token model. One is governance token without inflation while the other is utility token with inflation. With inflation people may hold it while without inflation people may spend it.

In this article, the problems in the dual token model will be discussed and a comparasion with single token model will be introduced. In the end, an interesting dual token economic model will be proposed.

*In the rest of this article, [neo](https://neo.org/) will be used as an example of explaination. We will follow the design of neo blockchain and use NEO to represent the governance token and GAS to represent the utility token.*

*In this article, it is focused on the token model instead of incentive model. So the detail of reward redistribution is ommited. For more detail about incentive model, see another article of mine (TODO: add link here).*

# The Problem

TODO: Difference between GP and NEO holder, Only GP get GAS

# The Solution

*Only the principles are statemented in this section. All the incentive details will be ommited here.*

## No Inflation on NEO

## Permanent Inflation on GAS

## Redistribution of Inflation GAS

The reward weight rate $r$ is defined in the following expression and it is used in the redistribution of inflation of GAS to evaluate how much weight per NEO the specific governance participant has. The specific governance participant owns $g$ GAS and $n$ NEO. Then an $\arctan$ function is attached to the proportion of GAS and NEO held by the participant and the proportion will be converged between $0$ and $\frac{\pi}{2}$. At last, the additional reward weight rate $S$ and the minimal reward weight rate $T$ is attached in a linear scale to adjust the basic reward rate and addtional reward rate.

$$
r = S \arctan{\frac{g}{n}} + T
$$

The following figure shows the relation between reward weight rate $r$ and proportion of GAS and NEO $\frac{g}{n}$. Obviously the reward rate grows when the proportion of GAS and NEO grows. The growth rate of reward weight rate $r$ decreases when proportion of GAS and NEO $\frac{g}{n}$ increases.

In [54]:
import numpy
import matplotlib
import ipywidgets

%matplotlib inline

S = ipywidgets.widgets.FloatSlider(min=0, max=4/numpy.pi, step=2/numpy.pi/10, value=2/numpy.pi)
T = ipywidgets.widgets.FloatSlider(min=0, max=2, step=0.1, value=1)

@ipywidgets.interact(S=S, T=T)
def draw(S, T):
    x = numpy.arange(0,100,0.001)
    y = S * numpy.arctan(x) + T
    matplotlib.pyplot.xlabel('g/n')
    matplotlib.pyplot.ylabel('r')
    matplotlib.pyplot.plot(x, y)
    matplotlib.pyplot.show()

interactive(children=(FloatSlider(value=0.6366197723675814, description='S', max=1.2732395447351628, step=0.06…

It would be more clear if logarithmic scale is used to plot the relation between reward weight rate $r$ and proportion of GAS and NEO $\frac{g}{n}$ in the follwing figure. It is pretty hard to increase reward weight rate $r$ a bit if proportion of GAS and NEO $\frac{g}{n}$ is enough large.

In [55]:
import numpy
import matplotlib
import ipywidgets

%matplotlib inline

S = ipywidgets.widgets.FloatSlider(min=0, max=4/numpy.pi, step=2/numpy.pi/10, value=2/numpy.pi)
T = ipywidgets.widgets.FloatSlider(min=0, max=2, step=0.1, value=1)

@ipywidgets.interact(S=S, T=T)
def draw(S, T):
    x = numpy.arange(-8,8,0.001)
    y = S * numpy.arctan(10**x) + T
    matplotlib.pyplot.xlabel('log(g/n)')
    matplotlib.pyplot.ylabel('r')
    matplotlib.pyplot.plot(x, y)
    matplotlib.pyplot.show()

interactive(children=(FloatSlider(value=0.6366197723675814, description='S', max=1.2732395447351628, step=0.06…

The reward weight $w$ is defined as the production of the number of NEO $n$ held by the participant and its reward weight rate $r$.

$$
w = n r
$$

The following 3D figure shows the relation of the number of NEO $n$ and the number of GAS $g$ held by the participant and its reward weight rate $r$.

In [56]:
import numpy
import matplotlib
import ipywidgets

%matplotlib inline

S = ipywidgets.widgets.FloatSlider(min=0, max=4/numpy.pi, step=2/numpy.pi/10, value=2/numpy.pi)
T = ipywidgets.widgets.FloatSlider(min=0, max=2, step=0.1, value=1)

@ipywidgets.interact(S=S, T=T)
def draw(S, T):
    x = numpy.arange(1,10,1)
    y = numpy.arange(0.1,100,0.1)
    x, y = numpy.meshgrid(x, y)
    z = x * S * numpy.arctan(10**(y/x)) + T
    matplotlib.pyplot.gca(projection='3d').plot_surface(x, y, z)
    matplotlib.pyplot.show()

interactive(children=(FloatSlider(value=0.6366197723675814, description='S', max=1.2732395447351628, step=0.06…

As a result, the reward $e$ in GAS can be counted as the following expression where $I$ is the amount of inflation GAS, $V$ is all the governance participants and $w_i$ is the reward weight of governance participant $i$.

$$
e = \frac{I w}{\sum_{i \in V}{w_i}}
$$

# The Analysis

TODO: SPEND ARCTAN

## More NEO Held, More GAS Rewarded

$$\frac{\partial e}{\partial n} \geqslant 0$$

## More GAS Held, More GAS Rewarded

$$\frac{\partial e}{\partial g} \geqslant 0$$

# The Experiment

TODO

## Retial Investor

Alice is a retial investor and she has totally some money for investment. She likes [neo](https://neo.org/) blockchain and she want to spend all her money to buy the native tokens on [neo](https://neo.org/) blockchain, a.k.a NEO and GAS. How many NEO coins should she buy and how many GAS coins should she buy? Which strategy should she use?

By assuming Alice has $A$ dollars and $N$ is the price of NEO in dollar and $G$ is the price of GAS in dollar, the following expression is statemented because Alice spend all her money to buy NEO and GAS.

$$
A = n N + g G
$$

Transform it into another form. The relation between $g$ and $n$ is shown:

$$
g = \frac{A - n N}{G}
$$

It is obvious that such relation is linear.

In [57]:
import numpy
import matplotlib
import ipywidgets

%matplotlib inline

S = ipywidgets.widgets.FloatSlider(min=0, max=4/numpy.pi, step=2/numpy.pi/10, value=2/numpy.pi)
T = ipywidgets.widgets.FloatSlider(min=0, max=2, step=0.1, value=1)

A = ipywidgets.widgets.FloatSlider(min=0, max=1000, step=10, value=100)
G = ipywidgets.widgets.FloatSlider(min=0, max=100, step=1, value=2)
N = ipywidgets.widgets.FloatSlider(min=0, max=100, step=1, value=10)

@ipywidgets.interact(S=S, T=T, A=A, G=G, N=N)
def draw(S, T, A, G, N):
    x = numpy.arange(0.001,A/N,0.001)
    y = (A - x * N) / G
    matplotlib.pyplot.xlabel('n')
    matplotlib.pyplot.ylabel('g')
    matplotlib.pyplot.plot(x, y)
    matplotlib.pyplot.show()

interactive(children=(FloatSlider(value=0.6366197723675814, description='S', max=1.2732395447351628, step=0.06…

Then the relation between reward weight rate $r$ and number of NEO $n$ held by Alice can be calculated.

$$
r = S \arctan{\frac{A - n N}{n G}} + T
$$

The more NEO Alice buys, the less GAS Alice buys, then, the less reward weight rate $r$.

In [58]:
import numpy
import matplotlib
import ipywidgets

%matplotlib inline

S = ipywidgets.widgets.FloatSlider(min=0, max=4/numpy.pi, step=2/numpy.pi/10, value=2/numpy.pi)
T = ipywidgets.widgets.FloatSlider(min=0, max=2, step=0.1, value=1)

A = ipywidgets.widgets.FloatSlider(min=0, max=1000, step=10, value=100)
G = ipywidgets.widgets.FloatSlider(min=0, max=100, step=1, value=2)
N = ipywidgets.widgets.FloatSlider(min=0, max=100, step=1, value=10)

@ipywidgets.interact(S=S, T=T, A=A, G=G, N=N)
def draw(S, T, A, G, N):
    x = numpy.arange(0.001,A/N,0.001)
    y = S * numpy.arctan((A - x * N) / (x * G)) + T
    matplotlib.pyplot.xlabel('n')
    matplotlib.pyplot.ylabel('r')
    matplotlib.pyplot.plot(x, y)
    matplotlib.pyplot.show()

interactive(children=(FloatSlider(value=0.6366197723675814, description='S', max=1.2732395447351628, step=0.06…

The reward weight $w$ can be evaluated by multiplying the number of NEO $n$ held by Alice.

$$
w = n S \arctan{\frac{A - n N}{n G}} + n T
$$

The following figure shows the relation between reward weight $w$ and number of NEO $n$ bought by Alice. At first, reward weight $w$ grows when $n$ grows but later reward weight $w$ goes down.

In [59]:
import numpy
import matplotlib
import ipywidgets

%matplotlib inline

S = ipywidgets.widgets.FloatSlider(min=0, max=4/numpy.pi, step=2/numpy.pi/10, value=2/numpy.pi)
T = ipywidgets.widgets.FloatSlider(min=0, max=2, step=0.1, value=1)

A = ipywidgets.widgets.FloatSlider(min=0, max=1000, step=10, value=100)
G = ipywidgets.widgets.FloatSlider(min=0, max=100, step=1, value=2)
N = ipywidgets.widgets.FloatSlider(min=0, max=100, step=1, value=10)

@ipywidgets.interact(S=S, T=T, A=A, G=G, N=N)
def draw(S, T, A, G, N):
    x = numpy.arange(0.001,A/N,0.001)
    y = x * S * numpy.arctan((A - x * N) / (x * G)) + x * T
    matplotlib.pyplot.xlabel('n')
    matplotlib.pyplot.ylabel('w')
    matplotlib.pyplot.plot(x, y)
    matplotlib.pyplot.show()

interactive(children=(FloatSlider(value=0.6366197723675814, description='S', max=1.2732395447351628, step=0.06…

By assuming the total supply of NEO is $U$, the rate of $NEO$ which participated in blockchain governance is $R$ and the average reward weight per NEO is $W$, we have $\sum_{i \in V}{w_i} = U R W$. Then the relation between reward $e$ in GAS and reward weight $w$ can be evaluated as the following expression:

$$
e = \frac{w I}{U R W}
$$

The relation between reward $e$ in GAS and amount of NEO $n$ bought by Alice can be shown in the following figure:

In [60]:
import numpy
import matplotlib
import ipywidgets

%matplotlib inline

S = ipywidgets.widgets.FloatSlider(min=0, max=4/numpy.pi, step=2/numpy.pi/10, value=2/numpy.pi)
T = ipywidgets.widgets.FloatSlider(min=0, max=2, step=0.1, value=1)

A = ipywidgets.widgets.FloatSlider(min=0, max=1000, step=10, value=100)
G = ipywidgets.widgets.FloatSlider(min=0, max=100, step=1, value=2)
N = ipywidgets.widgets.FloatSlider(min=0, max=100, step=1, value=10)

I = ipywidgets.widgets.FloatSlider(min=10**6, max=10**8, step=10**6, value=10**7)
U = ipywidgets.widgets.FloatSlider(min=10**8, max=10**8, step=10**8, value=10**8)
R = ipywidgets.widgets.FloatSlider(min=0, max=1, step=0.01, value=0.25)
W = ipywidgets.widgets.FloatSlider(min=0, max=2, step=0.01, value=0.4)

@ipywidgets.interact(S=S, T=T, A=A, G=G, N=N, I=I, U=U, R=R, W=W)
def draw(S, T, A, G, N, I, U, R, W):
    x = numpy.arange(0.001,A/N,0.001)
    y = x * S * numpy.arctan((A - x * N) / (x * G)) + x * T
    y = y * I / U / R / W
    matplotlib.pyplot.xlabel('n')
    matplotlib.pyplot.ylabel('e')
    matplotlib.pyplot.plot(x, y)
    matplotlib.pyplot.show()

interactive(children=(FloatSlider(value=0.6366197723675814, description='S', max=1.2732395447351628, step=0.06…

Then the rate of profit $p$ in dollar can be couted:

$$
p = \frac{e G}{A}
$$

And the relation between rate of profit $p$ and amount of NEO $n$ bought by Alice can be drawn:

In [61]:
import numpy
import matplotlib
import ipywidgets

%matplotlib inline

S = ipywidgets.widgets.FloatSlider(min=0, max=4/numpy.pi, step=2/numpy.pi/10, value=2/numpy.pi)
T = ipywidgets.widgets.FloatSlider(min=0, max=2, step=0.1, value=1)

A = ipywidgets.widgets.FloatSlider(min=0, max=1000, step=10, value=100)
G = ipywidgets.widgets.FloatSlider(min=0, max=100, step=1, value=2)
N = ipywidgets.widgets.FloatSlider(min=0, max=100, step=1, value=10)

I = ipywidgets.widgets.FloatSlider(min=10**6, max=10**8, step=10**6, value=10**7)
U = ipywidgets.widgets.FloatSlider(min=10**8, max=10**8, step=10**8, value=10**8)
R = ipywidgets.widgets.FloatSlider(min=0, max=1, step=0.01, value=0.25)
W = ipywidgets.widgets.FloatSlider(min=0, max=2, step=0.01, value=0.4)

@ipywidgets.interact(S=S, T=T, A=A, G=G, N=N, I=I, U=U, R=R, W=W)
def draw(S, T, A, G, N, I, U, R, W):
    x = numpy.arange(0.001,A/N,0.001)
    y = x * S * numpy.arctan((A - x * N) / (x * G)) + x * T
    y = y * I / U / R / W
    y = y * G / A
    matplotlib.pyplot.xlabel('n')
    matplotlib.pyplot.ylabel('p')
    matplotlib.pyplot.plot(x, y)
    matplotlib.pyplot.show()

interactive(children=(FloatSlider(value=0.6366197723675814, description='S', max=1.2732395447351628, step=0.06…

As a result, the best strategy for Alice is to buy $n$ NEO to reach the highest rate of profit $p$.

# The Conclusion

With the proposed solution in this article, the advantages of dual token economic model can be really activated. We hope the designers of blockchains can be inspired from this article and make their blockchain more sustainable.

# The Appendix

## Notation

- $A$: the total money in dollar owned by the specific participant of neo governance
- $e$: reward in GAS
- $G$: price of GAS in dollar
- $g$: amount of GAS held by the specific participant of neo governance
- $I$: amount of inflation GAS
- $N$: price of NEO in dollar
- $n$: amount of NEO held by the specific participant of neo governance
- $p$: rate of profit in dollar
- $r$: reward weight rate
- $S$: additional reward weight rate
- $T$: minimal reward weight rate
- $V$: the set of all governance participant
- $w$: reward weight

@vang1ong7ang